# Experimento TCC

## Libs

In [1]:
from sklearn.neighbors import (
    NeighborhoodComponentsAnalysis, 
    KNeighborsClassifier
)
from psutil import cpu_count
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


sns.set()

In [2]:
from modules.classNCA import NCA
from modules.optimization_funcs import PSO, Gradient
from modules.utils import run_experiments

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
columns_names = [
    'tau1','tau2','tau3',
    'tau4','p1','p2','p3',
    'p4','g1','g2','g3','g4'
]
dtypes = {i:float for i in columns_names}

EletricalGrid = pd.read_csv(
    'dataset/Data_for_UCI_named.csv',
    dtype=dtypes, nrows=300).drop('stab', axis=1)

In [5]:
X, y = EletricalGrid.drop('stabf',axis=1), EletricalGrid.stabf 

In [6]:
K_Folds = 5
K_neighbors = 5 # 50
max_iter= 10#1_000
N_cpu = cpu_count(logical=False)

In [7]:
experiments_configs = {
    'K_Folds':K_Folds,
    'nca_pso_options':{
        'X':X, 'y':y, 'K_Folds':K_Folds, 'model_class':NCA,
        'model_options':{
            'optimization_func':PSO, 'max_iter':max_iter, 
            'k':K_neighbors, 'cpu_count': N_cpu,
            'swarm_size':20
        }
    },
    'nca_gradient_options': {
        'X': np.asarray(X), 'y': np.asarray(y), 
        'K_Folds': K_Folds,'k': K_neighbors, 
        'model_class': NeighborhoodComponentsAnalysis,
        'model_options': {
            'n_components':None, 'max_iter':max_iter,
            'random_state':42
        }
    },
    'knn_options': {
        'X': np.asarray(X), 'y': np.asarray(y), 
        'K_Folds': K_Folds, 
        'model_options': {
            'n_neighbors': K_neighbors, 'n_jobs': N_cpu
        },
        'model_class': KNeighborsClassifier
    }
}

In [8]:
first_msg = f'''
{'='*30}
Execução de validação cruzada para NCA+PSO, NCA+Gradient e KNN
{'='*30}
Dimensões de X = {X.shape}
KFold = {K_Folds}
Número de vizinhos usados na classificação = {K_neighbors}
Número de núcleos sendo utilizados = {N_cpu}
'''
print(first_msg)


Execução de validação cruzada para NCA+PSO, NCA+Gradient e KNN
Dimensões de X = (300, 12)
KFold = 5
Número de vizinhos usados na classificação = 5
Número de núcleos sendo utilizados = 4



In [9]:
df = run_experiments(experiments_configs, n_runs = 2)

KNN: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  7.45it/s]

[{'KNN': {'acc': [0.7333333333333333, 0.75, 0.8833333333333333, 0.85, 0.75], 'mcc': [0.35355339059327373, 0.45630518910673146, 0.7516598260212388, 0.6579490028442165, 0.45657169762696875], 'partition_time': [0.03477954864501953, 0.024910688400268555, 0.02468705177307129, 0.02485036849975586, 0.024746179580688477]}}, {'KNN': {'acc': [0.7333333333333333, 0.75, 0.8833333333333333, 0.85, 0.75], 'mcc': [0.35355339059327373, 0.45630518910673146, 0.7516598260212388, 0.6579490028442165, 0.45657169762696875], 'partition_time': [0.028368234634399414, 0.02470874786376953, 0.025043249130249023, 0.025014162063598633, 0.024623394012451172]}}]
